In [ ]:
import os
import sys
sys.path.append('../')
from utils import visualize_images
from diffusers import StableDiffusionPipeline
from semantic_aug.augmentations.textual_inversion import load_embeddings
device = 'cuda:1'
os.environ["DISABLE_TELEMETRY"] = 'YES'
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5",local_files_only = True).to(device)

## Using the montage to showcase the generated samples of different fine-tuning strategies.

In [ ]:
from PIL import Image
import os
from utils import DATASET_NAME_MAPPING
import random
def create_image_grid(input_dir, output_path):
    image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
    random.shuffle(image_files)
    while len(image_files) < 8:
        image_files.append(None)

    grid_image = Image.new('RGB', (224*8, 224*1), (255, 255, 255))

    for i in range(8):
        row = i // 8
        col = i % 8

        if image_files[i] is not None:
            image_path = os.path.join(input_dir, image_files[i])
            img = Image.open(image_path)
            img = img.resize((224, 224), Image.ANTIALIAS)
            grid_image.paste(img, (col * 224, row * 224))
        else:
            grid_image.paste(Image.new('RGB', (224, 224), (255, 255, 255)), (col * 224, row * 224))

    grid_image.save(output_path)
    
def create_image_grid_for_ds(dataset_name, class_name, output_path):
    ds = DATASET_NAME_MAPPING[dataset_name]()
    target_indices=ds.label_to_indices[ds.class2label[class_name.replace('_', ' ')]]

    while len(target_indices) < 8:
        target_indices.append(None)

    grid_image = Image.new('RGB', (224*8, 224*1), (255, 255, 255))

    for i in range(8):
        row = i // 8
        col = i % 8

        if target_indices[i] is not None:
            img = ds.get_image_by_idx(target_indices[i])
            img = img.resize((224, 224), Image.ANTIALIAS)
            grid_image.paste(img, (col * 224, row * 224))
        else:
            grid_image.paste(Image.new('RGB', (224, 224), (255, 255, 255)), (col * 224, row * 224))

    grid_image.save(output_path)

In [ ]:
dataset_name='cub'
dirname = 'American_Three_toed_Woodpecker'
output_path = f'../figures/case_real_{dirname}.pdf'
create_image_grid_for_ds(dataset_name,dirname, output_path)

input_directory = f'../outputs/aug_samples/cub/real-generation-Multi5/data/{dirname}'
output_path = f'../figures/case_realgen_{dirname}.pdf'
create_image_grid(input_directory, output_path)
for strategy in ['db','ti','ti_db']:
    input_directory = f'../outputs/aug_samples/cub/dreambooth-lora-generation-Multi1_{strategy}_latest/data/{dirname}'
    output_path = f'../figures/case_{strategy}_{dirname}.pdf'
    create_image_grid(input_directory, output_path)

## Instance visualization

In [ ]:
import random
from utils import DATASET_NAME_MAPPING, AUGMENT, parse_finetuned_ckpt

gpu_id = 0
seed = 6

def synthesize_images(model, strength, train_dataset,source_label=1, target_label=2, source_image=None):
    num = 1
    random.seed(seed)
    target_indice = random.sample(train_dataset.label_to_indices[target_label], 1)[0]

    if source_image is None:
        source_indice = random.sample(train_dataset.label_to_indices[source_label], 1)[0]
        source_image = train_dataset.get_image_by_idx(source_indice)
    source_metadata = train_dataset.get_metadata_by_idx(source_indice)
    target_metadata = train_dataset.get_metadata_by_idx(target_indice)
    image, _ = model(image=[source_image], label=target_label, strength=strength, metadata=target_metadata)
    return image

In [ ]:

dataset = 'car' # 'cub' 'aircraft' 
aug = 'dreambooth-lora-mixup' #'dreambooth-lora-augmentation/mixup" "real-mixup" "real-gen"
finetune_model_key = 'ti_db_latest'
guidance_scale = 7
prompt = "a photo of a {name}"
train_dataset = DATASET_NAME_MAPPING[dataset](split="train")
lora_path, embed_path = parse_finetuned_ckpt(dataset=dataset, finetune_model_key=finetune_model_key)
AUGMENT[aug].pipe=None
model = AUGMENT[aug](
    embed_path=embed_path,
    lora_path=lora_path, 
    prompt=prompt, 
    guidance_scale=guidance_scale,
    mask=False, 
    inverted=False,
    device=f'cuda:{gpu_id}'
    )


In [ ]:
source_label = 1
target_label = 20
strength_list = [0.1, 0.3, 0.5, 0.7, 0.9]

image_list = []
for strength in strength_list:
    image = synthesize_images(model=model, strength=strength, train_dataset=train_dataset, source_label=source_label, target_label=target_label, source_image=None)
    image_list.append(image[0])

visualize_images(image_list,nrow=5,show=False,save=False,outpath='outputs/debug/aug_samples/diffmix-db-2.png')

In [ ]:
seed = 16
random.seed(seed)
source_label=9
target_label=9
source_indices = random.sample(train_dataset.label_to_indices[source_label], 5)
target_indice = random.sample(train_dataset.label_to_indices[target_label], 1)[0]

source_image = [train_dataset.get_image_by_idx(source_indice).resize((512,512)) for source_indice in source_indices]
visualize_images(source_image,nrow=5,show=False,save=False,outpath='outputs/debug/real_images.png')